In [9]:
eval_df.to_csv('/proj/NIRAL/users/siyangj/inference_eval_3.csv',index=None)

In [18]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 10000

In [2]:
def Dice(pred,true,labels=[0,1,2,3]):
    l = len(labels)
    ret = np.zeros(l)
    for i in range(l):
        lab = labels[i]
        p = pred==lab
        t = true==lab
        ret[i] = 2*np.logical_and(p,t).sum()/(p.sum()+t.sum())
    return ret

In [9]:
def inference_eval(model,inf_dir,data_path,eval_df,test_im,seg_suffix='seg/'):
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    
    inf_ims = {f[0]:os.path.join(inf_dir,f) for f in os.listdir(inf_dir) 
            if (os.path.isfile(os.path.join(inf_dir, f)) and im_re.match(f) is not None)}
    l = len(eval_df.index)
    if len(eval_df.loc[eval_df['inf_dir']==inf_dir])>0:
        l = np.where(eval_df['inf_dir']==inf_dir)[0][0]
    eval_df.loc[l,'inf_dir']=inf_dir
    eval_df.loc[l,'model']=model
    eval_df.loc[l,'test_im']=test_im
    for im_num,im in inf_ims.items():
        true_im = os.path.join(data_path,seg_suffix,im_num+'.nrrd')
        true_arr = sitk.GetArrayFromImage(sitk.ReadImage(true_im))
        #print(true_arr.shape)
        inf_arr = sitk.GetArrayFromImage(sitk.ReadImage(im))
        #print(inf_arr.shape)
        c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),inf_arr.flatten())
        p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
        eval_df.loc[l,im_num]=[p_mat.T]
        if im_num == test_im:
            eval_df.loc[l,'test_11'] = p_mat[1,1]
            eval_df.loc[l,'test_12'] = p_mat[2,1]
            eval_df.loc[l,'test_21'] = p_mat[1,2]
            eval_df.loc[l,'test_22'] = p_mat[2,2]
            eval_df.loc[l,'test_11+22'] = eval_df.loc[l,'test_11']+eval_df.loc[l,'test_22']
            eval_df.loc[l,'test_12+21'] = eval_df.loc[l,'test_12']+eval_df.loc[l,'test_21']
            Dice0123 = Dice(inf_arr,true_arr)
            eval_df.loc[l,'Dice0'] = Dice0123[0]
            eval_df.loc[l,'Dice1'] = Dice0123[1]
            eval_df.loc[l,'Dice2'] = Dice0123[2]
            eval_df.loc[l,'Dice3'] = Dice0123[3]

In [10]:
def model_eval(model,data_path,eval_df,test_im,seg_suffix='seg/'):
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    inf_dirs = [f for f in os.listdir(model_path) 
            if (os.path.isdir(os.path.join(model_path, f)) 
                and inf_re.match(f) is not None)]
    for d in inf_dirs:
        inf_dir = os.path.join(model_path,d)
        inference_eval(model,inf_dir,data_path,eval_df,test_im,seg_suffix=seg_suffix)

In [13]:
eval_df = pd.DataFrame(columns=['model','inf_dir','test_im','test_11','test_12','test_21','test_22',
                                'test_11+22','test_12+21',
                                'Dice0','Dice1','Dice2','Dice3',
                                '1','2','3','4','5','6','7','8'],dtype=object)

for i in range (1,9):
    eval_df[str(i)] = eval_df[str(i)].astype(object)

In [14]:
data_path = '/proj/NIRAL/users/siyangj/myData/min_cut_outside/'
models = ['10112050',
         #'10112055',
         '10112056',
         '10112103']
for m in models:
    model_eval(m,data_path,eval_df,'1')

models = ['10112127','10112130']
data_path = '/proj/NIRAL/users/siyangj/myData/min_fill_outside/'

for m in models:
    model_eval(m,data_path,eval_df,'1')

In [15]:
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1')

In [20]:
models = ['10192000','10192002','10192004']
data_path = '/proj/NIRAL/users/siyangj/myData/min_normal/'

for m in models:
    model_eval(m,data_path,eval_df,'1',seg_suffix='seg_inf_1/')

In [27]:
inference_eval('10192004','/proj/NIRAL/users/siyangj/model_10192004/inference_11100',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1',seg_suffix='seg_inf_1')

In [28]:
eval_df

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3,1,2,3,4,5,6,7,8
0,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_17000_10112050,1,0.918915,0.0673993,0.295044,0.649303,1.56822,0.362443,0.993768,0.780229,0.744064,0.805368,"[[[0.9999959575951733, 0.0, 0.0, 4.042404826631363e-06], [6.643598615916955e-05, 0.9189148788927...","[[[0.9928993426344548, 2.4038683789606042e-05, 0.002326204939024954, 0.00475041374273061], [0.00...","[[[0.9999966100890529, 0.0, 0.0, 3.3899109470394213e-06], [0.0, 0.9103068266146233, 0.0633052664...","[[[0.9999965429444089, 0.0, 0.0, 3.457055591182434e-06], [0.0, 0.929605867097402, 0.056333365103...","[[[0.9999257343727982, 0.0, 1.9042468513278313e-06, 7.236138035045758e-05], [0.0, 0.954769760031...","[[[0.9999572765882033, 0.0, 0.0, 4.272341179672517e-05], [0.0, 0.9691011449945531, 0.03014466697...","[[[0.9999670620785711, 0.0, 0.0, 3.293792142894114e-05], [0.0, 0.9581721725529981, 0.04021445832...","[[[0.9998296215910888, 0.0, 7.674703104110418e-06, 0.00016270370580714086], [0.0, 0.967761422740..."
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.75351,1.64869,0.321524,0.991811,0.803432,0.796687,0.804782,"[[[0.9999878727855201, 0.0, 0.0, 1.2127214479894088e-05], [6.643598615916955e-05, 0.895175086505...","[[[0.9928475670078311, 0.0, 0.00671788755443375, 0.00043454543773518615], [0.0001215395749429592...","[[[0.9999694908014767, 0.0, 1.0169732841118263e-05, 2.0339465682236526e-05], [0.0001986689182477...","[[[0.999970615027475, 0.0, 1.209969456913852e-05, 1.728527795591217e-05], [3.0097968888732824e-0...","[[[0.9998495644987451, 0.0, 2.6659455918589638e-05, 0.00012377604533630904], [0.0, 0.93974706031...","[[[0.9999177178735766, 0.0, 9.494091510383372e-06, 7.278803491293918e-05], [0.0, 0.9596547494800...","[[[0.9998557005346923, 0.0, 6.11704255108907e-05, 8.312903979685145e-05], [7.045280014654182e-06...","[[[0.9998894842753008, 0.0, 4.758315924548459e-05, 6.293256545370542e-05], [0.0, 0.9471481825281..."
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_13200_10112050,1,0.926572,0.0585024,0.308779,0.649881,1.57645,0.367281,0.994202,0.777533,0.752149,0.844704,"[[[0.9999818091782802, 0.0, 0.0, 1.8190821719841134e-05], [0.00011072664359861591, 0.92657162629...","[[[0.9957155668968833, 0.0, 0.003198994073539881, 0.0010854390295768268], [7.181883973902138e-05...","[[[0.9999762706233707, 0.0, 1.0169732841118263e-05, 1.3559643788157685e-05], [0.0002337281391149...","[[[0.9999879003054308, 0.0, 3.457055591182434e-06, 8.642638977956085e-06], [2.5081640740610688e-...","[[[0.9999257343727982, 0.0, 1.713822166195048e-05, 5.712740553983494e-05], [0.0, 0.9326481121168...","[[[0.9999335413594274, 0.0, 3.1646971701277907e-06, 6.329394340255581e-05], [0.0, 0.962206799881...","[[[0.9998964808183662, 0.0, 7.842362244985987e-06, 9.567681938882903e-05], [2.8181120058616728e-...","[[[0.9999263228502006, 0.0, 1.3814465587398752e-05, 5.986268421206126e-05], [7.36713374294597e-0..."
3,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_18100_10112056,1,0.825539,0.124662,0.174211,0.737446,1.56298,0.298872,0.992718,0.794706,0.796743,0.786574,"[[[0.9999979787975867, 0.0, 0.0, 2.0212024133156815e-06], [0.0001328719723183391, 0.825539100346...","[[[0.9902920700080438, 7.396518089109552e-06, 0.005569578121099493, 0.004130955352767684], [4.97...","[[[1.0, 0.0, 0.0, 0.0], [3.505922086724826e-05, 0.804077387386861, 0.09069820438357125, 0.105189...","[[[1.0, 0.0, 0.0, 0.0], [5.016328148122138e-06, 0.8629789966340438, 0.0721247661137001, 0.064891...","[[[0.9999695320503788, 0.0, 9.521234256639156e-06, 2.0946715364606144e-05], [0.0, 0.907509234977...","[[[0.9999841765141494, 0.0, 0.0, 1.5823485850638954e-05], [0.0, 0.9557695384216901, 0.0415031957...","[[[0.999990589165306, 0.0, 0.0, 9.410834693983183e-06], [0.0, 0.9418412134790297, 0.053198909390...","[[[0.999930927672063, 1.5349406208220835e-06, 6.13976248328833

In [25]:
eval_df.loc[23,'8']

[array([[9.99631784e-01, 0.00000000e+00, 7.26524587e-05, 2.95563412e-04],
        [0.00000000e+00, 9.18632933e-01, 7.94124347e-02, 1.95463196e-03],
        [5.36210379e-04, 4.91233423e-02, 9.29030707e-01, 2.13097401e-02],
        [7.52538807e-03, 7.26914726e-03, 1.16400761e-01, 8.68804704e-01]])]

In [20]:
test_33 = [r['1'][0][3][3] for i,r in eval_df.iterrows()]

In [21]:
test_33

[0.7861897343557165,
 0.7174722317761547,
 0.8239828433851943,
 0.793766564832986,
 0.8083840815748478,
 0.8598134622777025,
 0.8622311691990208,
 0.8547049183644567,
 0.7924110304084812,
 0.75131001274607,
 0.8554231492908734,
 0.8064620551520424,
 0.9158882972112544,
 0.8747693572596297,
 0.903756250059752]

In [22]:
eval_df.insert(loc=9,column='test_33',value=test_33)

In [24]:
eval_df.sort_values('test_11+22',ascending=False).iloc[:,0:10]

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,test_33
6,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal,1,0.861763,0.124861,0.121004,0.822459,1.68422,0.245865,0.862231
5,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.12516,0.120896,0.821308,1.68232,0.246056,0.859813
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.75351,1.64869,0.321524,0.717472
8,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_16500_10112056,1,0.793855,0.189398,0.0904256,0.849576,1.64343,0.279823,0.792411
9,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference,1,0.845575,0.147383,0.184313,0.789043,1.63462,0.331696,0.751310
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference,1,0.824332,0.159098,0.141585,0.805546,1.62988,0.300683,0.793767
11,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference_12300_10112103,1,0.809434,0.176941,0.13089,0.809682,1.61912,0.307831,0.806462
10,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference,1,0.721373,0.2495,0.0527411,0.869897,1.59127,0.302241,0.855423
4,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055,1,0.892019,0.0944388,0.250173,0.696974,1.58899,0.344611,0.808384
13,10112127,/proj/NIRAL/users/siyangj/model_10112127/inference_8700_10112127,1,0.713201,0.271862,0.0509308,0.864683,1.57788,0.322792,0.874769
